<a href="https://www.kaggle.com/code/thiagosalesfreireluz/exoplanet-extremely-trees?scriptVersionId=224345076" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import sys
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from numpy.random import seed
import time


In [46]:
data = pd.read_csv('../input/kepler-exoplanet-search-results/cumulative.csv')

In [47]:
# define the model
#model = ExtraTreesClassifier(n_estimators = 200, max_features = 'sqrt', max_depth = None, criterion = 'gini')


In [48]:
# getting data
df = data

# Drop unused columns
df = df.drop(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_pdisposition', 'koi_score'], axis=1)
  
# Limit target values to CANDIDATE and CONFIRMED
false_positive_rows = df.query("koi_disposition == 'FALSE POSITIVE'").index
df = df.drop(false_positive_rows, axis=0).reset_index(drop=True)

#Transforming target column in binary data
df['koi_disposition'] = df['koi_disposition'].map({"CANDIDATE":1,"CONFIRMED":0})
    
# Drop columns with all missing values
df = df.drop(['koi_teq_err1', 'koi_teq_err2'], axis=1)

# Fill remaining missing values
df['koi_tce_delivname'] = df['koi_tce_delivname'].fillna(df['koi_tce_delivname'].mode()[0])
for column in df.columns[df.isna().sum() > 0]:
    df[column] = df[column].fillna(df[column].mean())
    
# One-hot encode koi_tce_delivname column
delivname_dummies = pd.get_dummies(df['koi_tce_delivname'], prefix='delivname')
df = pd.concat([df, delivname_dummies], axis=1)
df = df.drop('koi_tce_delivname', axis=1)

# Split df into X and y
y = df['koi_disposition']
X = df.drop('koi_disposition', axis=1)


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
# Scale X
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
X_train, X_test, y_train, y_test

(      koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  koi_period  \
 1673      -0.071134      -0.117116        -0.0355      -0.017742   -0.037165   
 1239      -0.071134      -0.117116        -0.0355      -0.017742   -0.004275   
 3589      -0.071134      -0.117116        -0.0355      -0.017742    0.174458   
 415       -0.071134      -0.117116        -0.0355      -0.017742   -0.037962   
 2925      -0.071134      -0.117116        -0.0355      -0.017742    0.010035   
 ...             ...            ...            ...            ...         ...   
 2895      -0.071134      -0.117116        -0.0355      -0.017742   -0.035650   
 2763      -0.071134      -0.117116        -0.0355      -0.017742   -0.031798   
 905       -0.071134      -0.117116        -0.0355      -0.017742   -0.026434   
 3980      -0.071134      -0.117116        -0.0355      -0.017742   -0.034204   
 235       -0.071134      -0.117116        -0.0355      -0.017742   -0.037964   
 
       koi_period_err1  ko

In [49]:
# define the model
#clf = ExtraTreesClassifier()
clf = ExtraTreesClassifier(n_estimators = 200, max_features = 'sqrt', max_depth = None, criterion = 'entropy')

In [50]:
#clf = model.fit(X_train, y_train)

In [51]:
def get_classifications(y_test, y_pred, positive_label=1):
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    
    for y_t, y_p in zip(y_test, y_pred):
        if y_t == positive_label:
            if y_p == positive_label:
                tp += 1
            else:
                fn += 1
        else:
            if y_p == positive_label:
                fp += 1
            else:
                tn += 1
    
    return tp, fn, fp, tn

def get_accuracy(tp, fn, fp, tn):
    acc = (tp + tn) / (tp + fn + fp + tn)
    return acc

def get_precision(tp, fn, fp, tn):
    precision = tp / (tp + fp)
    return precision

def get_recall(tp, fn, fp, tn):
    recall = tp / (tp + fn)
    return recall

def get_f1_score(tp, fn, fp, tn):
    precision = get_precision(tp, fn, fp, tn)
    recall = get_recall(tp, fn, fp, tn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


def get_sensitivity(tp, fn, fp, tn):
    sensitivity = tp / (tp + fn)
    return sensitivity

def get_specificity(tp, fn, fp, tn):
    specificity = tn / (tn + fp)
    return specificity

In [52]:
seed(0)

In [53]:
# Measure time
start_time = time.time()

In [54]:
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
n_scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print(n_scores)
# report performance
print('Accuracy: %.4f (%.4f)' % (mean(n_scores), std(n_scores)))

[0.8021978  0.84581498 0.81938326 0.83700441 0.83700441 0.80837004
 0.83039648 0.83920705 0.82378855 0.79735683 0.82637363 0.83920705
 0.79735683 0.84140969 0.82599119 0.81497797 0.83700441 0.82599119
 0.83039648 0.79735683 0.82857143 0.84361233 0.8215859  0.81057269
 0.81277533 0.81057269 0.83480176 0.82378855 0.80837004 0.87004405
 0.83956044 0.81938326 0.81938326 0.83700441 0.80176211 0.83039648
 0.7907489  0.84140969 0.83920705 0.79295154 0.83296703 0.82378855
 0.83039648 0.83039648 0.78634361 0.82819383 0.83700441 0.84140969
 0.81497797 0.79515419]
Accuracy: 0.8235 (0.0172)


In [55]:
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
n_scores = cross_val_score(clf, X, y, scoring='recall', cv=cv, n_jobs=-1, error_score='raise')
print(n_scores)
# report performance
print('sensitivity: %.4f (%.4f)' % (mean(n_scores), std(n_scores)))

[0.74222222 0.83482143 0.78125    0.83555556 0.80888889 0.79555556
 0.77333333 0.79555556 0.74222222 0.76888889 0.77777778 0.79464286
 0.71875    0.8        0.81333333 0.77777778 0.78222222 0.78222222
 0.79555556 0.76       0.8        0.78125    0.83035714 0.72888889
 0.80888889 0.75111111 0.78666667 0.80444444 0.8        0.84
 0.8        0.80357143 0.76785714 0.78222222 0.82666667 0.79555556
 0.76       0.80444444 0.82222222 0.76444444 0.79555556 0.75892857
 0.84821429 0.8        0.76       0.80444444 0.82666667 0.79111111
 0.76888889 0.76444444]
sensitivity: 0.7891 (0.0284)


In [56]:
specificity = make_scorer(recall_score, pos_label=0)

In [57]:
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
n_scores = cross_val_score(clf, X, y, cv=cv, scoring = specificity, n_jobs=-1, error_score='raise')
print(n_scores)
# report performance
print('specificity: %.4f (%.4f)' % (mean(n_scores), std(n_scores)))

[0.88695652 0.86956522 0.83913043 0.82532751 0.84716157 0.84279476
 0.88646288 0.86462882 0.86026201 0.82532751 0.85217391 0.8826087
 0.86086957 0.84716157 0.84279476 0.87336245 0.88646288 0.86462882
 0.86026201 0.83406114 0.83913043 0.89130435 0.86521739 0.86462882
 0.84279476 0.87336245 0.86899563 0.87336245 0.81659389 0.86899563
 0.87391304 0.83913043 0.87391304 0.88646288 0.80786026 0.88209607
 0.84716157 0.89519651 0.86026201 0.84716157 0.89565217 0.8826087
 0.82173913 0.88209607 0.80349345 0.85152838 0.86899563 0.89082969
 0.83842795 0.83842795]
specificity: 0.8589 (0.0232)


In [58]:
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
n_scores = cross_val_score(clf, X, y, scoring='precision', cv=cv, n_jobs=-1, error_score='raise')
print(n_scores)
# report performance
print('precision: %.4f (%.4f)' % (mean(n_scores), std(n_scores)))

[0.84693878 0.84862385 0.83098592 0.8209607  0.82727273 0.82629108
 0.87373737 0.85781991 0.85       0.81990521 0.86956522 0.86792453
 0.83248731 0.84018265 0.82300885 0.85427136 0.84951456 0.85784314
 0.83796296 0.83732057 0.83710407 0.88265306 0.8440367  0.84422111
 0.83568075 0.85786802 0.8647343  0.83962264 0.81363636 0.86160714
 0.85116279 0.81944444 0.84729064 0.87192118 0.8        0.88442211
 0.83414634 0.85446009 0.84792627 0.82075472 0.86956522 0.8622449
 0.82378855 0.86764706 0.80930233 0.84976526 0.84545455 0.87192118
 0.8254717  0.8277512 ]
precision: 0.8448 (0.0195)


In [59]:
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
n_scores = cross_val_score(clf, X, y, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
print(n_scores)
# report performance
print('f1 score: %.4f (%.4f)' % (mean(n_scores), std(n_scores)))

[0.80285036 0.84738041 0.81278539 0.83296214 0.82696629 0.80995475
 0.8179669  0.81860465 0.8        0.79445727 0.81006865 0.83449883
 0.76923077 0.82352941 0.82326622 0.81775701 0.82790698 0.8271028
 0.80549199 0.7908046  0.82326622 0.82577566 0.82696629 0.79334917
 0.81018519 0.80285036 0.81922197 0.80821918 0.80630631 0.85714286
 0.83900227 0.8        0.81775701 0.82790698 0.80701754 0.82903981
 0.7908046  0.84331797 0.82993197 0.79445727 0.8287037  0.8056872
 0.83296214 0.8287037  0.7706422  0.80909091 0.83218391 0.82629108
 0.7954023  0.79452055]
f1 score: 0.8154 (0.0180)


In [60]:
# y_pred = clf.predict(X_test)
# print(" Accuracy: {:.3f}%".format(get_accuracy(*get_classifications(y_test, y_pred)) * 100))
# y_pred = clf.predict(X_test)
# print(" Sensitivity: {:.3f}%".format(get_sensitivity(*get_classifications(y_test, y_pred)) * 100))
# y_pred = clf.predict(X_test)
# print(" Specificity: {:.3f}%".format(get_specificity(*get_classifications(y_test, y_pred)) * 100))
# y_pred = clf.predict(X_test)
# print(" Precision: {:.3f}%".format(get_precision(*get_classifications(y_test, y_pred)) * 100))
# y_pred = clf.predict(X_test)
# print(" F1 Score: {:.5f}".format(get_f1_score(*get_classifications(y_test, y_pred))))

In [61]:
# Print Time
print("--- %s seconds ---" % (time.time() - start_time))

--- 155.58047223091125 seconds ---


In [62]:
y_pred = clf.predict(X_test)
cf_matrix = confusion_matrix(y_test, y_pred)
group_names = ['True Neg','False Pos','False Neg','True Pos']

group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)

ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

#ax.set_title('Confusion Matrix with labels\n\n');
ax.set_xlabel('Valores preditos pelo modelo')
ax.set_ylabel('Valores reais ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('my_plot.JPEG')
plt.show()

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
print(n_estimators)

max_features = ['auto', 'sqrt', 'log2']

# Maximum number of levels in tree
max_depth = [4,5,6,7,8]
max_depth.append(None)
print (max_depth)

criterion = ['gini', 'entropy']


In [ ]:
# Exemplo dos parâmetros que quero testar.
parametros = {'n_estimators': n_estimators,
            'max_features': max_features,
            'max_depth' : max_depth,
            'criterion' : criterion}

In [ ]:
from sklearn.model_selection import  RandomizedSearchCV

In [ ]:
random = RandomizedSearchCV(estimator = clf, 
                   param_distributions = parametros,  scoring = 'accuracy',
                   cv = 10, n_iter = 100)

In [ ]:
import time
start_time = time.time()

In [ ]:
# fit the model
random.fit(X_train, y_train)

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Imprimindo os resultados.
pd.DataFrame(random.cv_results_)

In [ ]:
# Imprime os parâmetros que produziram o ".best_score_".
random.best_params_

In [ ]:
# Imprimindo o score.
random.best_score_

In [ ]:
# Imprime colunas de interesse.
pd.DataFrame(random.cv_results_)[['params','rank_test_score','mean_test_score']]